In [ ]:
# first of all, let's check that avro tools work

!java -jar /usr/bin/avro.jar

In [ ]:
# check that Spark client works

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .config(
            "spark.sql.extensions",
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )
        .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.iceberg.type", "rest")
        .config("spark.sql.catalog.iceberg.uri", "http://nessie:19120/iceberg/main/")
        .config("spark.sql.catalog.iceberg.ref", "main")
        .config("spark.sql.catalog.iceberg.cache-enabled", False)
        .getOrCreate()
)
spark.sparkContext.setLogLevel('ERROR')

spark.sql("""
    SHOW TABLES FROM iceberg.default
""").show()

In [ ]:
# check that Trino client works

from trino.dbapi import connect

trino_connection = connect(
    host="trino",
    port=8080,
    user="trino",
)
trino = trino_connection.cursor()
trino.execute("SELECT * FROM system.runtime.nodes")

rows = trino.fetchall()
print(rows)

In [ ]:
# check that Python client works

from pyiceberg.catalog.rest import RestCatalog
from pyiceberg.exceptions import NoSuchTableError


catalog = RestCatalog(name="iceberg", uri="http://nessie:19120/iceberg/main/")

try:
    catalog.load_table("default.sample")
except NoSuchTableError as e:
    print(e)